In [2]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import trange
from gymnasium.utils.env_checker import check_env
from gymnasium import spaces

from flygym import Fly, SingleFlySimulation, Camera, Parameters

from flygym.examples.common import PreprogrammedSteps
from flygym.arena import FlatTerrain
from flygym import preprogrammed
import dm_control.mjcf as mjcf
from flygym.vision import save_video_with_vision_insets
from typing import Union, Tuple, List, Optional

In [7]:
fly = Fly(
    enable_adhesion =True,
    draw_adhesion=True
)

arena= FlatTerrain()

# Define simulation parameters
sim_params = Parameters(
    timestep=1e-4,
    render_mode="saved",
    render_playspeed=0.3,
    render_window_size=(800, 608),
    enable_olfaction=True,
    enable_adhesion=True,
    draw_adhesion=False,
    render_camera="Animat/camera_right",
)

timestep = sim_params.timestep

cam = Camera(
    fly=fly,
    camera_id="Animat/camera_right",
    window_size=(800, 608),
    play_speed=0.5,
)

sim= SingleFlySimulation(fly=fly, cameras= cam, arena=arena)

In [8]:
preprogrammed_steps = PreprogrammedSteps()
swing_periods = preprogrammed_steps.swing_period
legs = preprogrammed_steps.legs

adhesion_action= np.zeros(len(legs))
# Initialize standing action using preprogrammed steps for all legs
standing_action = []
for leg in legs:
    standing_action.extend(
        preprogrammed_steps.get_joint_angles(leg, swing_periods[leg][1])
    )

print(standing_action[fly.actuated_joints.index("joint_LHCoxa")],)
stand_action = {"joints": standing_action, "adhesion": adhesion_action}

timestep = sim.timestep

# Let the fly stand on the floor first
for i in range(int(0.2 // timestep)):
    sim.step(stand_action)

run_time1 =0.2
run_time2 =0.5

# Simulation parameters
target_num_steps1 = int(run_time1 / timestep)
target_num_steps2 = int(run_time2 / timestep)

# Define group actions for the specific joints
group_action = {
    "joint_LMTibia": np.linspace(standing_action[fly.actuated_joints.index("joint_LMTibia")], 2.60, target_num_steps1),
    "joint_RMTibia": np.linspace(standing_action[fly.actuated_joints.index("joint_RMTibia")], 2.60, target_num_steps1),
    "joint_LMTarsus1": np.linspace(standing_action[fly.actuated_joints.index("joint_LMTarsus1")], -1.6, target_num_steps1),
    "joint_RMTarsus1": np.linspace(standing_action[fly.actuated_joints.index("joint_RMTarsus1")], -1.6, target_num_steps1),
    "joint_LHTarsus1": np.linspace(standing_action[fly.actuated_joints.index("joint_LHTarsus1")], -1, target_num_steps1),
    "joint_RHTarsus1": np.linspace(standing_action[fly.actuated_joints.index("joint_RHTarsus1")], -1, target_num_steps1),
    "joint_LHCoxa": np.linspace(standing_action[fly.actuated_joints.index("joint_LHCoxa")], 0.2, target_num_steps1),
    "joint_RHCoxa": np.linspace(standing_action[fly.actuated_joints.index("joint_RHCoxa")], 0.2, target_num_steps1),
    "joint_LHTibia": np.linspace(standing_action[fly.actuated_joints.index("joint_LHTibia")], standing_action[fly.actuated_joints.index("joint_LHTibia")], target_num_steps1),
    "joint_RHTibia": np.linspace(standing_action[fly.actuated_joints.index("joint_RHTibia")], standing_action[fly.actuated_joints.index("joint_RHTibia")], target_num_steps1)
}

#define the action for the jump
jumps_action = {
    "joint_LMTibia": np.linspace(2.60, 0, target_num_steps2),
    "joint_RMTibia": np.linspace(2.60, 0, target_num_steps2),
    "joint_LMTarsus1": np.linspace(-1.6, 0, target_num_steps2),
    "joint_RMTarsus1": np.linspace(-1.6, 0, target_num_steps2),
    "joint_LHTarsus1": np.linspace(-1, 0, target_num_steps2),
    "joint_RHTarsus1": np.linspace(-1, 0, target_num_steps2),
    "joint_LHCoxa": np.linspace(0.2, 0, target_num_steps2),
    "joint_RHCoxa": np.linspace(0.2, 0, target_num_steps2),
    "joint_LHTibia": np.linspace(standing_action[fly.actuated_joints.index("joint_LHTibia")], 1.5,target_num_steps2),
    "joint_RHTibia": np.linspace(standing_action[fly.actuated_joints.index("joint_RHTibia")], 1.5,target_num_steps2)

}


# Actuated joints and data block initialization
actuated_joints = ["joint_LMTibia", "joint_RMTibia", "joint_LMTarsus1", "joint_RMTarsus1" ,"joint_LHTarsus1", "joint_RHTarsus1","joint_LHCoxa", "joint_RHCoxa", "joint_LHTibia", "joint_RHTibia"]
data_block1 = np.zeros((len(actuated_joints), target_num_steps1))
output_t1 = np.arange(target_num_steps1) * timestep
input_t1 = np.linspace(0, timestep * (target_num_steps1 - 1), num=target_num_steps1)

data_block2 = np.zeros((len(actuated_joints), target_num_steps2))
output_t2 = np.arange(target_num_steps2) * timestep
input_t2 = np.linspace(0, timestep * (target_num_steps2 - 1), num=target_num_steps2)


# Interpolation for the specified joints
for i, joint in enumerate(actuated_joints):
    data_block1[i, :] = np.interp(output_t1, input_t1, np.array(group_action[joint]))

for i, joint in enumerate(actuated_joints):
    data_block2[i, :] = np.interp(output_t2, input_t2, np.array(jumps_action[joint]))


# Get the total number of actuated joints
total_actuated_joints = len(fly.actuated_joints)

# Lists to store joint angle values for plotting
joint_LMTibia_values = []
joint_RMTibia_values = []
joint_LMTarsus_values = []
joint_RMTarsus_values = []
joint_LHTarsus_values = []
joint_RHTarsus_values = []
joint_LHCoxa_values = []
joint_RHCoxa_values = []


joint_angles=standing_action.copy()

# Main loop to update joint angles and perform actions
for i in range(target_num_steps1):
    # Update specific joints with interpolated values
    joint_angles[fly.actuated_joints.index("joint_LMTibia")] = data_block1[0, i]
    joint_angles[fly.actuated_joints.index("joint_RMTibia")] = data_block1[1, i]
    joint_angles[fly.actuated_joints.index("joint_LMTarsus1")] = data_block1[2, i]
    joint_angles[fly.actuated_joints.index("joint_RMTarsus1")] = data_block1[3, i]
    joint_angles[fly.actuated_joints.index("joint_LHTarsus1")] = data_block1[4, i]
    joint_angles[fly.actuated_joints.index("joint_RHTarsus1")] = data_block1[5, i]    
    joint_angles[fly.actuated_joints.index("joint_LHCoxa")] = data_block1[6, i]
    joint_angles[fly.actuated_joints.index("joint_RHCoxa")] = data_block1[7, i]
    joint_angles[fly.actuated_joints.index("joint_LHTibia")] = data_block1[8,i]
    joint_angles[fly.actuated_joints.index("joint_RHTibia")] = data_block1[9,i]


    # Store values for plotting
    joint_LMTibia_values.append(data_block1[0, i])
    joint_RMTibia_values.append(data_block1[1, i])
    joint_LMTarsus_values.append(data_block1[2, i])
    joint_RMTarsus_values.append(data_block1[3, i])
    joint_LHTarsus_values.append(data_block1[4, i])
    joint_RHTarsus_values.append(data_block1[5, i])
    joint_LHCoxa_values.append(data_block1[6, i])
    joint_RHCoxa_values.append(data_block1[7, i])

    action = {"joints": joint_angles, "adhesion": adhesion_action}

    sim.step(action)
    sim.render()

for i in range (target_num_steps2):
    if i < (target_num_steps2 - 2000):
        # If it's the last iteration, suppress adhesion
        adhesion_action = np.ones(len(legs))
    else:
        # Otherwise, keep adhesion enabled
        adhesion_action = np.zeros(len(legs))
        # Update specific joints with interpolated values
        
    joint_angles[fly.actuated_joints.index("joint_LMTibia")] = data_block2[0, i]
    joint_angles[fly.actuated_joints.index("joint_RMTibia")] = data_block2[1, i]
    joint_angles[fly.actuated_joints.index("joint_LMTarsus1")] = data_block2[2, i]
    joint_angles[fly.actuated_joints.index("joint_RMTarsus1")] = data_block2[3, i]
    joint_angles[fly.actuated_joints.index("joint_LHTarsus1")] = data_block2[4, i]
    joint_angles[fly.actuated_joints.index("joint_RHTarsus1")] = data_block2[5, i]   
    joint_angles[fly.actuated_joints.index("joint_LHCoxa")] = data_block2[6, i]
    joint_angles[fly.actuated_joints.index("joint_RHCoxa")] = data_block2[7, i]
    joint_angles[fly.actuated_joints.index("joint_LHTibia")] = data_block2[8,i]
    joint_angles[fly.actuated_joints.index("joint_RHTibia")] = data_block2[9,i]

    # Store values for plotting
    joint_LMTibia_values.append(data_block2[0, i])
    joint_RMTibia_values.append(data_block2[1, i])
    joint_LMTarsus_values.append(data_block2[2, i])
    joint_RMTarsus_values.append(data_block2[3, i])
    joint_LHTarsus_values.append(data_block2[4, i])
    joint_RHTarsus_values.append(data_block2[5, i])
    joint_LHCoxa_values.append(data_block2[6, i])
    joint_RHCoxa_values.append(data_block2[7, i])

    action = {"joints": joint_angles, "adhesion": adhesion_action}

    sim.step(action)
    sim.render()

cam.save_video('v1.mp4')

from IPython.display import Video
Video('v1.mp4', width=800, height=608)



0.37979870389165027
